In [1]:
import pandas as pd
from db import DB
from datetime import datetime
import numpy as np
import json
import re
import multiprocessing
from gensim.models.phrases import Phraser, Phrases
from gensim.models.word2vec import LineSentence
from gensim.models.doc2vec import  Doc2Vec, LabeledSentence
import itertools as it
import spacy
from os.path import dirname, basename, abspath, join
import pandas as pd

nlp = spacy.load('en')

In [2]:
%%time

def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def get_sentences(parsed):
    sentences = []
    for sent in parsed.sents:
        sentences.append( [token.text for token in list(sent)])
    return sentences
domains =set()


CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 9.06 µs


In [5]:
%%time 
def get_article(filename):
    with open(filename) as readfile:
        count =0 
        for i, line in enumerate(readfile):
#             if i > 5000:
#                 break
            data = json.loads(line)
            yield "{}. {}".format(data.get('title',''),data.get('text'))

def lemmatized_sentence_corpus(filename):            
    for parsed_doc in nlp.pipe(get_article(filename), batch_size=1000, n_threads=8):
        for sent in parsed_doc.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

if False:
    with open("models/fc_unigram_sentences.txt", 'w') as f:
        for sentence in lemmatized_sentence_corpus("finance_articles.json"):
            f.write(sentence + '\n')
    
    # Create BIGRAMS
    unigram_sentences = LineSentence("models/fc_unigram_sentences.txt")
    bigram_model = Phraser(Phrases(unigram_sentences))
    bigram_model.save("models/fc_bigram.model")
    print("Created bigram model")
    
    with open("models/fc_bigram_sentences.txt", 'w') as f:
        for unigram_sentence in unigram_sentences:
            sent = " ".join(bigram_model[unigram_sentence])
            f.write(sent + '\n')
    print("Created bigrams")
            
    # Create TRIGRAMS
    bigram_sentences = LineSentence("models/fc_bigram_sentences.txt")
    trigram_model = Phraser(Phrases(bigram_sentences))
    trigram_model.save("models/fc_trigram.model")
    print("Created trigram model")
    
    with open("models/fc_trigram_sentences.txt", 'w') as f:
        for bigram_sentence in bigram_sentences:
            sent = " ".join(trigram_model[bigram_sentence])
            f.write(sent + '\n')
    print("Created trigrams")
   

Created bigram model
Created bigrams
Created trigram model
Created trigrams
CPU times: user 58min 47s, sys: 12.9 s, total: 59min
Wall time: 59min 6s


In [6]:
%%time
if False:
    with open("finance_articles_trigrams.txt", 'w') as writefile:
        for parsed_doc in nlp.pipe(get_article('finance_articles.json'), batch_size=1000, n_threads=8):
         # lemmatize the text, removing punctuation and whitespace
            unigram_doc = [token.lemma_ for token in parsed_doc
                              if not punct_space(token)]

            # apply the first-order and second-order phrase models
            bigram_doc = bigram_model[unigram_doc]
            trigram_doc = trigram_model[bigram_doc]

            # remove any remaining stopwords
            trigram_doc = [term for term in trigram_doc
                              if term not in spacy.en.language_data.STOP_WORDS]

            # write the transformed review as a line in the new file
            trigram_doc = u' '.join(trigram_doc)

            writefile.write(trigram_doc + '\n')
        

CPU times: user 53min 13s, sys: 8.46 s, total: 53min 21s
Wall time: 53min 27s


In [9]:
%%time 
with open('finance_articles.json') as fc, \
    open('finance_articles_trigrams.txt') as fcn,\
    open('finance_articles_trigrams.json', 'w') as writefile:
    count=0
    for line, trigram in zip(fc, fcn):
        count +=1
        data = json.loads(line)
        data['trigrams_text'] = trigram
        writefile.write("{}\n".format(json.dumps(data)))
        

http://www.bankrate.com/finance/auto/5-steps-to-cutting-a-big-car-repair-bill-1.aspx
5 steps to cutting a big car repair bill
5 step cut big car repair bill day car owner -PRON- toyota_ford lexus brand local auto shop receive unexpected expensive news -PRON- ne major car repair work cost several_hundred thousand dollar -PRON- perfect world car repair place consumer trust -PRON- -PRON- familiar_with shop ’s service say_john_nielsen director auto_repair buy service aaa ’s national office heathrow_fla. -PRON- visit shop regularly oil change_tire_rotation minor maintenance establish relationship -PRON- -PRON- approve -PRON- estimate confidence -PRON- -PRON- ’re like driver -PRON- probably sense trust -PRON- local mechanic ’s shop situation -PRON- ’s important weigh option expensive decision potential money save step auto expert car owner -PRON- receive major car repair estimate bankrate help -PRON- find good auto_loan_rate 5 money save step ask_probe_question check shop ’s qualification pr

https://www.nerdwallet.com/blog/credit-cards/received-chip-card-what-now/
You’ve Received Your New Chip Card — What Now?
-PRON- receive -PRON- new chip_card time -PRON- unexpectedly receive new credit_card mail -PRON- probably -PRON- bank worried_about recent security_breach time -PRON- ’re new card -PRON- card_issuer adopt international standard type fraud hard pull -PRON- haven’t receive chip_card -PRON- probably hit -PRON- mailbox week ’s -PRON- expect -PRON- receive -PRON- new card emv_chip_card credit_card microprocessor embed -PRON- use emv_technology company develop -PRON- europay_mastercard visa emv_card handle transaction differently traditional card store datum magnetic_stripe -PRON- ’s hard thief skim private information emv_card hard -PRON- duplicate -PRON- card commit_fraud improved security -PRON- use emv_chip transaction create unique dynamic datum jon krauss manage card reissue team discover transaction data steal criminal -PRON- replicate reuse way magstripe transactio

https://www.thebalance.com/u-s-army-garrison-usag-benelux-brussels-3344654
U.S. Army Garrison Benelux-Brussels Information
u.s._army_garrison benelux_brussels information usag_benelux_brussels formerly_know_as nato support activity u.s._army_garrison brussels originally activate august 1967 nato headquarters paris brussels shape begin function belgium u.s._army designate executive agency support u.s. government_entity include department defense non dod agency assign brussels area usag_benelux_brussels mean u.s._army exercise responsibility primary_mission usag_benelux_brussels provide base operation support quality life service u.s. personnel family assign u.s. representation nato headquarters include active_duty service member assign u.s. nato permrep milrep staff department defense department state civilian u.s. personnel_assign nato international military international staffs official website u.s._army_garrison beneluxuusag benelux_brussels locate northeastern brussels 5 minute driv

https://www.nerdwallet.com/blog/banking/banking-news/events-financial-literacy-programs-march/
Spotlight: Financial Literacy Programs in March
spotlight financial_literacy programs march -PRON- spend -PRON- free time month finalize march_madness_bracket enjoy hint spring weather let financial education fall wayside numerous opportunity age demographic learn manage -PRON- personal_finance read great event come month financial education day westchester community_college valhalla ny second annual financial education day host westchester community_college participant able attend variety workshop topic such_as retirement_planning money management build economic security addition cnn fox_news_anchor kiran chetry event ’s keynote_speaker free event place saturday march_23 8:30am 1:00pm financial_literacy boot_camp workshop parents family centered educational agency inc. chicago_il come beginning month april 6th 11:00am 1:00pm fcea host free financial_literacy_workshop specifically_design pare

### Dump articles from Elastic search into file.

In [7]:
%%time
from elasticsearch import Elasticsearch, helpers
HOSTS = "http://search-datascience-dev-eugrfsatyqi2pffna6ajnk6i5a.us-east-1.es.amazonaws.com:80"
client = Elasticsearch(hosts=HOSTS)
query = {"query": {"terms": {"domain":['www.bankrate.com','www.nerdwallet.com', 'thefinancebase.com',  
    'www.sapling.com', 'www.gobankingrates.com','www.thebalance.com','www.creditcards.com',
    'www.thepointsguy.com','www.simpledollar.com' ]}}}

# with open('finance_articles.json','w') as write_file:
    for i, r in enumerate(helpers.scan(client=client, query=query, index="competitors", size=1000, scroll='1h', request_timeout=1000)):
        article = r['_source']
        if 'topics' in article and len(article.get('text','')) > 200: 
            write_file.write(json.dumps(article)+"\n")
        if i%1000 == 0:
            print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
CPU times: user 57 s, sys: 20.2 s, total: 1min 17s
Wall time: 14min 34s


In [19]:
%%time 

with open('finance_articles.json') as readfile:
    for i, line in enumerate(readfile):

        data = json.loads(line)
        if 'www.gobankingrates.com' not in data.get('url'): continue
        common = set(['Savings account']).intersection( set([topic.get('label') for topic in data.get('topics') if topic.get('score') > 0.85]))
        if len(common) ==1:
            print(data.get('url'))
            print([topic.get('label') for topic in data.get('topics') if topic.get('score') > 0.85])
            print("\n\n")
    

www.gobankingrates.com/savings-account/go-banking-rates-study-average-savings-account-rate-state/
['Bank', 'Credit union', 'Interest', 'Interest rate', 'United States', 'Economy', 'Banking', 'Financial services', 'Finance', 'Money', 'Savings account', 'U.S. state', 'Service industries']



www.gobankingrates.com/avings Interest Rates Deal: 1.00% APY at Highmark Credit Union/
['Credit union', 'Interest', 'Interest rate', 'Adolescence', 'Financial services', 'Finance', 'Economy', 'Savings account', 'Service industries', 'Money']



www.gobankingrates.com/banking/announcing-best-banks-2016/
['Bank', 'Transaction account', 'Online banking', 'Interest', 'Savings account', 'Interest rate', 'Deposit account', 'Bank of Internet USA', 'First Niagara Bank', 'Certificate of deposit', 'PNC Financial Services', 'Capital One', 'Citibank', 'Fee', 'Federal Deposit Insurance Corporation', 'Investment', 'Bank of America', 'First National Bank of Omaha', 'Credit (finance)', 'Annual percentage yield', 'Se

www.gobankingrates.com/banking/redwood-credit-union-bank-review-deposit-loan-products/
['Transaction account', 'Credit (finance)', 'Savings account', 'Loan', 'Mortgage loan', 'Credit card', 'Economy', 'Investment', 'Business', 'Credit', 'Service industries', 'Finance', 'Financial services', 'Banking', 'Money', 'Debt', 'Credit union', 'Bank']



www.gobankingrates.com/savings-account/losangeles-california/la-financial-helps-members-achieve-financial-goals/
['Transaction account', 'Credit (finance)', 'Financial services', 'Finance', 'Economy', 'Money', 'Banking', 'Service industries', 'Business', 'Savings account', 'Credit union', 'Fee', 'Bank', 'Economic institutions']



www.gobankingrates.com/banking/sanfrancisco-california/california-ranks-6-worst-state-affordable-banking-sanfrancisco/
['Transaction account', 'Bank', 'Debits and credits', 'Savings account', 'Money', 'Finance', 'Banking', 'Financial services', 'Business', 'Economy', 'Service industries', 'Fee', 'Payments', 'Cheque']



www.gobankingrates.com/savings-account/best-savings-account-rates-state-average/
['Bank', 'U.S. state', 'Savings account', 'Interest rate', 'Interest', 'Credit union', 'Financial services', 'Finance', 'United States', 'Economy', 'Banking', 'Deposit account', 'Washington, D.C.', 'Service industries', 'Union (American Civil War)']



www.gobankingrates.com/savings-account/fnbo-savings-account-review-great-rate/
['Saving', 'Savings account', 'Bank', 'Interest', 'Online and offline', 'Money', 'First National Bank of Omaha', 'Deposit account', 'User (computing)', 'Interest rate', 'Fee', 'Online savings account', 'Service industries', 'Finance', 'Economy', 'Financial services', 'Business', 'Banking', 'Investment', 'Market (economics)', 'Certificate of deposit']



www.gobankingrates.com/cd-rates/cd-rates-savings-account-rates-study-fallen-2011/
['Certificate of deposit', 'Savings account', 'Interest', 'Bank', 'Deposit account', 'Interest rate', 'Finance', 'Economy', 'Money', 'Banking', 'Fina

www.gobankingrates.com/cd-rates/houston-texas/deal-of-the-day-the-mint-national-bank-cd-rates-at-1-00-apy/
['Interest', 'Investment', 'Service industries', 'Economy', 'Finance', 'Financial services', 'Banking', 'Money', 'Certificate of deposit', 'Savings account', 'Debt']



www.gobankingrates.com/savings-account/td-bank/
['TD Banknorth', 'Bank', 'Savings account', 'Finance', 'Financial services', 'Banking', 'Business', 'Service industries', 'Money', 'Economy', 'Federal Deposit Insurance Corporation', 'Fee', 'Economy of the United States', 'Investment', 'Economic institutions']



www.gobankingrates.com/savings-account/study-shows-boston-savings-rates-remain-abysmally-low/
['Bank', 'Interest rate', 'Credit union', 'Savings account', 'Credit (finance)', 'Finance', 'Federal Reserve System', 'Interest', 'Money', 'Financial services', 'Economy', 'Banking', 'Service industries', 'Business', 'Boston', 'Credit', 'Financial market', 'Cooperative banking']



www.gobankingrates.com/savings-acco

www.gobankingrates.com/savings-account/wichita-kansas/wichita-ranks-29-city-saving-money/
['Savings account', 'Money', 'Real estate appraisal', 'Government', 'Economics', 'Economy of the United States', 'Tax', 'Government finances', 'Finance', 'Economy', 'Politics', 'Unemployment', 'Wichita, Kansas', 'United States', 'Economies']



www.gobankingrates.com/savings-account/coloradosprings-colorado/save-enough-money-buy-house-colorado-springs/
['Money', 'Savings account', 'Certificate of deposit', 'Financial services', 'Service industries', 'Banking', 'Investment', 'Economy', 'Finance', 'Deposit account', 'Transaction account', 'Mortgage loan', 'Interest']



www.gobankingrates.com/banking/10-cd-accounts-2015/
['Certificate of deposit', 'Bank', 'Interest', 'Deposit account', 'Interest rate', 'Financial services', 'Finance', 'Service industries', 'Economy', 'Banking', 'Money', 'Business', 'Savings account', 'Investment', 'Online banking']



www.gobankingrates.com/banking/video5-reasons-ne

...

www.gobankingrates.com/savings-account/build-savings-account-during-america-saves-week/
['Credit (finance)', 'Debt', 'Bank', 'Credit union', 'Savings account', 'United States', 'Transaction account', 'Credit', 'Service industries', 'Financial services', 'Banking', 'Money', 'Economy', 'Finance', 'Loan', 'Credit score', 'Wells Fargo', 'Interest', 'Liability (financial accounting)', 'Credit card', 'Mortgage loan']



www.gobankingrates.com/savings-account/savings-tips-comparing-online-high-yield-savings-accounts/
['High-yield debt', 'Interest', 'Interest rate', 'Savings account', 'Money', 'Service industries', 'Debt', 'Economics', 'Banking', 'Financial services', 'Economy', 'Finance', 'Investment', 'Macroeconomics', 'Monetary economics', 'Financial markets', 'Yield (finance)']



www.gobankingrates.com/savings-account/finding-right-savings-account/
['Bank', 'Savings account', 'Finance', 'Interest', 'Service industries', 'Banking', 'Financial services', 'Money', 'Economy', 'Interest rate',

www.gobankingrates.com/banking/banking-fees-should-never-pay/
['Overdraft', 'Transaction account', 'Certificate of deposit', 'Fee', 'Bank', 'ATM usage fees', 'Cheque', 'Financial transaction', 'Credit card', 'Debits and credits', 'Automated teller machine', 'Debit card', 'Wire transfer', 'Economy', 'Financial technology', 'Payments', 'Business', 'Banking', 'Financial services', 'Finance', 'Money', 'Service industries', 'Savings account', 'Non-sufficient funds']



www.gobankingrates.com/savings-account/boulder-colorado/7-goals-can-achieve-savings-account-boulder/
['Money', 'Investment', 'Finance', 'Economy', 'Employment', 'Paycheck', 'Tuition payments', 'Savings account']



www.gobankingrates.com/personal-finance/tips-tricks-always-pay-yourself-first/
['Transaction account', 'Debit card', 'Bank', 'Credit card', 'Tax refund', 'Debits and credits', 'Money', 'Financial transaction', 'Economy', 'Service industries', 'Financial services', 'Banking', 'Payments', 'Finance', 'Investment', 'Pa

www.gobankingrates.com/savings-account/march-madness-2014-sweet-16-saving-money/
['Bank', 'Savings account', 'Debits and credits', 'Deposit account', 'Bank of America', 'Interest', 'Credit union', 'Debit card', 'Citibank', 'Transaction account', 'Business', 'Money', 'Economy', 'Service industries', 'Financial services', 'Finance', 'Banking', 'Chase Bank', 'Automated teller machine', 'Interest rate']



www.gobankingrates.com/banking/10-savings-accounts-2015/
['Savings account', 'Bank', 'Transaction account', 'Deposit account', 'Cheque', 'Fee', 'Interest', 'Wire transfer', 'Finance', 'Banking', 'Financial services', 'Money', 'Service industries', 'Economy', 'Business', 'Online banking', 'Overdraft', 'Online savings account', 'Payments', 'Interest rate']



www.gobankingrates.com/savings-account/detroit-michigan/michigans-flagstar-bank-savings-account-ranked-2015/
['Bank', 'Deposit account', 'Savings account', 'Transaction account', 'Certificate of deposit', 'Online banking', 'Investment

www.gobankingrates.com/banking/10-bank-accounts-beat-inflation/
['Inflation', 'Interest', 'Bank', 'Investment', 'Interest rate', 'Deflation', 'Savings account', 'Finance', 'Money', 'Economics', 'Financial services', 'Economy', 'Federal Reserve System', 'Credit union', 'Business', 'Macroeconomics', 'Market liquidity', 'Banking', 'Monetary economics']



www.gobankingrates.com/savings-account/experts-share-secrets-17-money-saving-tips/
['Bank', 'Money', 'Credit card', 'Fee', 'Finance', 'Personal finance', 'Interest rate', 'Economy', 'Advertising', 'Savings account', 'Service industries', 'Automated teller machine', 'Credit (finance)', 'Coupon', 'Financial services', 'Banking']



www.gobankingrates.com/savings-account/why-online-savings-account-rates-better-cds/
['Certificate of deposit', 'Interest', 'Bank', 'Federal Reserve System', 'Savings account', 'Interest rate', 'Money', 'Investment', 'Banking', 'Economy', 'Finance', 'Debt', 'Financial services', 'Economics', 'Financial markets', 

www.gobankingrates.com/savings-account/sanantonio-texas/alamo-federal-credit-union-050-apy/
['Savings account', 'Credit union', 'Interest rate', 'Government', 'Economy', 'Finance', 'Service industries', 'Business', 'Financial services', 'Employment', 'Money']



www.gobankingrates.com/savings-account/saltlakecity-utah/first-utah-bank-savings-interest-rates-0-75/
['Bank', 'Interest', 'Interest rate', 'Business', 'Money', 'Economy', 'Finance', 'Banking', 'Financial services', 'Service industries', 'Savings account']



www.gobankingrates.com/banking/difference-between-community-bank-credit-unions/
['Bank', 'Transaction account', 'Credit union', 'Credit (finance)', 'National Credit Union Administration', 'Savings account', 'Interest', 'Certificate of deposit', 'Loan', 'Fee', 'Tax', 'Credit card', 'Online banking', 'Saving', 'Business', 'Service industries', 'Economic institutions', 'Banking', 'Investment', 'Money', 'Financial services', 'Economy', 'Finance', 'S corporation', 'Nonprofit or